In [ ]:
# 11/9/2021-1
# https://m.youtube.com/watch?v=Zrt76AIbeh4&noapp=1&noapp=1
import os
from os.path import *
from BashColors import C
import numpy as np
import tensorflow
from tensorflow import keras
# import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
# from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

contentPath=os.getcwd()
imgPath=join(contentPath, 'friendly_validation_12.png')

In [ ]:
def prepare_image(file):
    # img_path = 'MobileNet-inference-images/'
    img = image.load_img(file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
mobile = keras.applications.mobilenet.MobileNet()

layerCount=0
for layer in mobile.layers:
    # print(layer)
    layerCount+=1
# layerCount
print(f'{layerCount} layers')
print(f'{C.BIBlue}\n')
mobile.summary()

In [ ]:
from IPython.display import Image
Image(filename='test_0071.jpg', width=224,height=224)

In [ ]:
preprocessed_image = prepare_image('test_0071.jpg')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)

for result in results:
    print(result)

In [ ]:
from IPython.display import Image
Image(filename='enemy.png', width=224,height=224)

In [ ]:
preprocessed_image = prepare_image('enemy.png')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)

for result in results:
    print(result)

In [ ]:
x=mobile.layers[-6].output
output=Dense(units=2, activation='softmax')(x)

In [ ]:
model=Model(mobile.inputs, output)

In [ ]:
for layer in model.layers[:-23]:
    # print(layer)
    layer.trainable=False

In [ ]:
print(f'{C.Green}')
model.summary()